# Setup and Import Library

In [1]:
!pip install -q -U langchain langchain-core datasets sentence-transformers rank_bm25 bitsandbytes
!pip install -q -U gdown
!pip install -U -q langchain-community
!pip install -q -U "qdrant-client" "colpali_engine" "datasets" "huggingface_hub[hf_transfer]" "transformers" "pdf2image" "bitsandbytes"
# !sudo apt-get install poppler-utils
!pip install -q -U ragas
!pip install -q -U langchain-google-genai
!pip install -U transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 108.8/108.8 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.2/515.2 kB 22.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 494.1/494.1 kB 44.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.4/157.4 kB 15.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.6/47.6 MB 15.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 41.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 52.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.0/51.0 kB 5.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires reques

In [2]:
from google.colab import drive
import gdown
import os
import json

In [3]:
import os
import json
import shutil
from langchain_core.documents import Document
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import HuggingFaceEmbeddings

In [4]:
from colpali_engine.models import ColPali, ColPaliProcessor
from datasets import load_dataset
from qdrant_client import QdrantClient, models
import torch
from tqdm import tqdm
import uuid
import numpy as np
import random
import os
from pdf2image import convert_from_path
import matplotlib.pyplot as plt
from transformers import BitsAndBytesConfig
from langchain_core.prompts import PromptTemplate
from PIL import Image

In [5]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM, AutoModelForCausalLM, AutoModelForImageTextToText, BitsAndBytesConfig, AutoProcessor
import torch

In [6]:
from datasets import Dataset
from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, LLMContextPrecisionWithoutReference
from ragas import evaluate, RunConfig

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)
/tmp/ipython-input-550044629.py:2: DeprecationWarning: Importing LLMContextRecall from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Example: from ragas.metrics.collections import LLMContextRecall
  from ragas.metrics import LLMContextRecall, Faithfulness, FactualCorrectness, LLMContextPrecisionWithoutReference
/tmp/ipython-input-550044629.py:2: DeprecationWarning: Importing Faithfulness from 'ragas.metrics' is deprecated and will be removed in v1.0. Please use 'ragas.metrics.collections' instead. Examp

In [7]:
# Load quantization 4bit
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

# Load LMM Model

In [ ]:
from huggingface_hub import login
login(token = "YOUR_HUGGINGFACEHUB_TOKEN_HERE")

In [9]:
model_name = "google/gemma-3-4b-it"

In [10]:
llm_pipeline = pipeline(
    "image-text-to-text",
    model=model_name,
    # torch_dtype=torch.bfloat16,
    device="cuda:0",
    # quantization_config=bnb_config,
)

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/90.6k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/883 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

...

The image processor of type `Gemma3ImageProcessor` is now loaded as a fast processor by default, even if the model checkpoint was saved with a slow processor. This is a breaking change and may produce slightly different outputs. To continue using the slow processor, instantiate this class with `use_fast=False`. 


tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

In [11]:
messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "url": "https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG"},
            {"type": "text", "text": "What animal is on the candy?"}
        ]
    },
]
llm_pipeline(text=messages)

Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


[{'input_text': [{'role': 'user',
    'content': [{'type': 'image',
      'url': 'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG'},
     {'type': 'text', 'text': 'What animal is on the candy?'}]}],
  'generated_text': [{'role': 'user',
    'content': [{'type': 'image',
      'url': 'https://huggingface.co/datasets/huggingface/documentation-images/resolve/main/p-blog/candy.JPG'},
     {'type': 'text', 'text': 'What animal is on the candy?'}]},
   {'role': 'assistant',
    'content': 'Based on the image, the animal on the candy is a **turtle**. You can see the distinct shell shape of a turtle on the surface of the orange candy.'}]}]

# Load Gemini API

In [ ]:
import os
os.environ["GOOGLE_API_KEY"] = "YOUR_API_KEY"  # From https://ai.google.dev/

In [13]:
config = {
    "model": "gemma-3-27b-it",  # or other model IDs
    "temperature": 0.4,
    "max_tokens": None,
    "top_p": 0.8
}

In [14]:
from ragas.llms import LangchainLLMWrapper
from ragas.embeddings import LangchainEmbeddingsWrapper

# Choose the appropriate import based on your API:
from langchain_google_genai import ChatGoogleGenerativeAI

# Initialize with Google AI Studio
evaluator_llm = LangchainLLMWrapper(ChatGoogleGenerativeAI(
    model=config["model"],
    temperature=config["temperature"],
    max_tokens=config["max_tokens"],
    top_p=config["top_p"],
))

/tmp/ipython-input-2646173014.py:8: DeprecationWarning: LangchainLLMWrapper is deprecated and will be removed in a future version. Use llm_factory instead: from openai import OpenAI; from ragas.llms import llm_factory; llm = llm_factory('gpt-4o-mini', client=OpenAI(api_key='...'))
  evaluator_llm = LangchainLLMWrapper(ChatGoogleGenerativeAI(


In [15]:
# Google AI Studio Embeddings
from langchain_google_genai import GoogleGenerativeAIEmbeddings

evaluator_embeddings = LangchainEmbeddingsWrapper(GoogleGenerativeAIEmbeddings(
    model="models/embedding-001",  # Google's text embedding model
    task_type="retrieval_document"  # Optional: specify the task type
))

/tmp/ipython-input-1044670269.py:4: DeprecationWarning: LangchainEmbeddingsWrapper is deprecated and will be removed in a future version. Use the modern embedding providers instead: embedding_factory('openai', model='text-embedding-3-small', client=openai_client) or from ragas.embeddings import OpenAIEmbeddings, GoogleEmbeddings, HuggingFaceEmbeddings
  evaluator_embeddings = LangchainEmbeddingsWrapper(GoogleGenerativeAIEmbeddings(


# Evaluation with RAGAS

In [16]:
# Load questions and ground-truth
import json

def load_qa_data(json_file_path):
    questions = []
    ground_truths = []
    reasoning = []
    images = []

    try:
      with open(json_file_path, 'r', encoding='utf-8') as f:
          data = json.load(f)

          if not isinstance(data, list):
              print(f"Error: Expected JSON file to contain a list of objects.")
              return None, None

          for item in data:
              if 'question_clinical_vignette' in item and 'answer' in item and 'analytic_chain_of_thought' in item:
                  questions.append(item['question_clinical_vignette'])
                  ground_truths.append(item['answer'])
                  reasoning.append(item['analytic_chain_of_thought'])
                  if 'image' in item:
                      images.append(item['image'])

              else:
                  print(f"Warning: Skipping item, missing 'prompt' or 'diagnosis': {item}")

    except FileNotFoundError:
      print(f"Error: JSON file not found at {json_file_path}")
      return None, None
    except json.JSONDecodeError:
      print(f"Error: Could not decode JSON from {json_file_path}")
      return None, None

    return questions, ground_truths,reasoning, images

In [17]:
import os
import gdown

# https://drive.google.com/file/d/1dpUMPdTu9o5XmU9Pv_Vmy8Udtk9Htkz3/view?usp=sharing
# Google Drive file ID from the sharing link, then change the ID
file_id = "1dpUMPdTu9o5XmU9Pv_Vmy8Udtk9Htkz3"
# URL for downloading
url = f"https://drive.google.com/uc?id={file_id}"

output_path = "diagnosis.json"

# Download the file
gdown.download(url, output_path, quiet=False)

print(f"Downloaded to {output_path}")


Downloading...
From: https://drive.google.com/uc?id=1dpUMPdTu9o5XmU9Pv_Vmy8Udtk9Htkz3
To: /content/diagnosis.json
100%|██████████| 187k/187k [00:00<00:00, 51.0MB/s]

Downloaded to diagnosis.json


In [18]:
json_path = "/content/diagnosis.json"
questions, ground_truths, reasoning, images = load_qa_data(json_path)

In [19]:
len(questions)

93

In [20]:
i = 18
questions = [questions[i]]
ground_truths = [ground_truths[i]]

In [21]:
if torch.cuda.is_available():
        torch.cuda.empty_cache()

In [22]:
def get_answer_from_question_context(questions, images=None):
    answers = []
    prompt_template = """
    Question:
    {question}

    Answer:
    """

    prompt = PromptTemplate(template=prompt_template, input_variables=["question"])

    for i, (question, image) in enumerate(zip(questions, images if images else [None] * len(questions))):
        formatted_prompt = prompt.format(question=question)
        content = [{"type": "text", "text": formatted_prompt}]

        if image:
            content.append({"type": "image", "image": image})

        messages = [
            {
                "role": "system",
                "content": [
                    {
                        "type": "text",
                        "text": """You are MedGemma, a medical diagnosis reasoning assistant.
Given a patient's clinical case description, identify the **most likely diagnosis** concisely.
- Use both clinical and epidemiological clues.
- Provide **only the disease name** (no explanation, no extra text).
- If multiple related conditions fit, choose the **most specific and probable one**.
- If uncertain, give your **best single diagnosis guess**."""
                    }
                ]
            },
            {
                "role": "user",
                "content": content
            },
        ]

        result = llm_pipeline(text=messages, return_full_text=False)
        generated_answer = result[0]["generated_text"]
        answers.append(generated_answer)

    return answers


In [23]:
answers = get_answer_from_question_context(questions)

Both `max_new_tokens` (=256) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


In [24]:
len(answers)

1

In [25]:
answers

['Visceral leishmaniasis']

In [26]:
ground_truths

['Acute Schistosomiasis (Katayama Syndrome)']

In [27]:
if not (len(questions) == len(answers) == len(ground_truths)):
  print("Error: Lengths of questions, contexts, answers, and ground_truths do not match.")

dataset = []
for question, ground_truth, answer in zip(questions, ground_truths, answers):
  dataset.append({
      "user_input": question,
      "response": answer,
      "reference": ground_truth
  })

from ragas import EvaluationDataset
evaluation_dataset = EvaluationDataset.from_list(dataset)

save_path = "/content/eval_dataset_disease.json"
with open(save_path, "w") as f:
    json.dump(dataset, f, indent=2, ensure_ascii=False)

print(f"Dataset saved successfully at {save_path}")

Dataset saved successfully at /content/eval_dataset_disease.json


In [28]:
import json
from ragas import EvaluationDataset

with open('/content/eval_dataset_disease.json', 'r') as file:
    dataset = json.load(file)

evaluation_dataset = EvaluationDataset.from_list(dataset)

In [29]:
# Metrics: Context Precision, Context Recall, Faithfulness, Factual Correctness
# Context Precision: Measures the relevance of the context provided to the model.
# Context Recall: Measures how well the model utilizes the provided context in its answers.
# Faithfulness: Assesses whether the model's answers are supported by the provided context.
# Factual Correctness: Evaluates the factual accuracy of the model's answers.

result = evaluate(
    dataset=evaluation_dataset,
    metrics=[
             FactualCorrectness()
             ],
    llm=evaluator_llm,
    run_config=RunConfig(timeout=300)
)
result

Evaluating:   0%|          | 0/1 [00:00<?, ?it/s]

{'factual_correctness(mode=f1)': 0.0000}